What makes a good receipie?
first we need to import the revelevant libraries 

In [4]:
import matplotlib.pyplot as plt
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup  
import pandas as pd
import scipy as sc
import numpy as np
import requests
import re
import time
import csv



now we want to start scraping we will first get all the receipie names and links and save them to a file

In [23]:
def get_full_page_thespruceeats():#this returns a list of all the links to receipies on the page:
    # URL to scrape
    url = "https://www.thespruceeats.com/search?q=&searchType=recipe"

    # Configure the Selenium webdriver
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # Run in headless mode (no GUI)
    driver = webdriver.Chrome(options=options)
    driver.get(url)

    # Wait for the page to load
    wait = WebDriverWait(driver, 10)

    # Get the page source and parse it with BeautifulSoup
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "html.parser")

    results_div = soup.find("div", attrs={"class": "results-list__container"})
    recipe_names = []
    recipe_links = []

    # Scrape the first page
    for li in results_div.find_all("li", class_="results__item"):
        if li.find("a") is not None:
            link = li.find("a").get("href")
        else:
            link = ''

        if li.find("h4", class_="card__title") is not None:
            name = li.find("h4", class_="card__title").text.strip()
        else:
            name = ''
        
        recipe_names.append(name)
        recipe_links.append(link)

    # Scrape subsequent pages if the "Next" button exists
    while True:
        try:
            next_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".pagination__item-link--next")))
            next_button.click()
            time.sleep(5)

            # Get the page source and parse it with BeautifulSoup
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, "html.parser")

            results_div = soup.find("div", attrs={"class": "results-list__container"})

            # Scrape recipe names and links
            for li in results_div.find_all("li", class_="results__item"):
                if li.find("a") is not None:
                    link = li.find("a").get("href")
                else:
                    link = ''

                if li.find("h4", class_="card__title") is not None:
                    name = li.find("h4", class_="card__title").text.strip()
                else:
                    name = ''
                print(f"{name} added")
                recipe_names.append(name)
                recipe_links.append(link)

        except:
            break

    # Create a DataFrame with the recipe names and links
    df = pd.DataFrame({"Recipe_name": recipe_names, "Recipe_link": recipe_links})

    # Write DataFrame to a CSV file
    df.to_csv("Recipe_Links_and_Names.csv", index=False)

    # Close the driver
    driver.quit()

    print("Done!")

lets run this function!. we save this to a csv to speed up work later as we are no longer depenadant on the websriver in another place and also in case of internet lose during the next test

In [24]:
get_full_page_thespruceeats()

Simple Fresh Berry With Grand Marnier Sauce added
Pumpkin Swirl Cheesecake added
Graham Cracker Pie Crust added
Marinated Tofu Jerky added
Oven-Roasted Vegetables added
Moroccan Black-Eyed Peas (Cowpeas) - Ful Gnaoua added
Fried Fish Sandwich With Tartar Sauce and Hand-Cut Chips added
Brandied Peaches added
Totchos added
Sweet and Tropical Pineapple Smoothie Recipe added
Grilled Chicken Banh Mi Recipe added
Chinese Stir-Fry Scallop With Vegetables Recipe added
Pepper Steak Stir-Fry Recipe added
Grilled Salmon Burgers With Radicchio Slaw and Sambal Mayonnaise added
Kentucky Buck Cocktail Recipe added
Sparkling Borage Cocktail added
Chocolate Cherry Bread added
Lunch Box-Worthy Falafel Kebabs added
Chipotle Pumpkin Queso Dip added
Flamin' Hot Cheetos Mac and Cheese Bites added
Cherry Vinegar Recipe added
Turkish Ramadan Flat Bread (Pide) added
Carrot, Cabbage, and Kohlrabi Slaw With Miso Dressing Recipe added
Esquites Recipe (Mexican Corn Off the Cob) added
Gillie Fix Cocktail Recipe add

now that we have the receipie links we need to scrape each one we will break this into steps as much as possible first we need to soup object

In [26]:
def load_soup_object(url):
    ###
    #url = "https://www.thespruceeats.com/search?q=&searchType=recipe"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    return soup
    ###

next we will gather information by the order it is represented in our site using functions because we will loop over it soon!

our first function will gather the cook time prep time total time to cook and the amount of servings
this was very messy because of the formating can change widly from one recepie to another time could look like 25 min or 2h 15 min or 2 hours 10 min or countless other variations that why we used a complex regex expression to combat this issue 

In [7]:
def get_info1_fast(soup_obj):
    lines=[]
    results_items = soup_obj
    results_items = soup_obj.find_all(class_='comp article__decision-block mntl-block')
    if(results_items==[]):
        soup = soup_obj
        results_items = soup.find_all(class_='comp project-meta')        

    for item in results_items:
        item.find_all(class_='meta-text__data')
        for sub_item in item:
            if bool(sub_item.text.strip()):
                clean_text = sub_item.text.strip().replace('\n', '')
                lines.append(clean_text)

    if(len(lines)>1):
        new_string = lines[0] + lines[1]
        lines[0]= new_string

    #use regex expressions to clean up the line we get it looks something like this
    #['Prep: 15 minsCook: 20 minsTotal: 35 minsServings: 6 servingsYield: 1 cake', 'ratingsAdd a comment']
    #prep = re.findall(r'Prep:\s*(\d+)\s*mins', lines[0])[0]
    cook_time_str = re.findall(r'Cook:\s*(?:(\d+)\s*(?:hrs?|hours?)\s*)?(?:(\d+)\s*mins?)?', lines[0])[0]
    prep_time_str = re.findall(r'Prep:\s*(?:(\d+)\s*(?:hrs?|hours?)\s*)?(?:(\d+)\s*mins?)?', lines[0])[0]    
    total_time_str = re.findall(r'Total:\s*(?:(\d+)\s*(?:hrs?|hours?)\s*)?(?:(\d+)\s*mins?)?', lines[0])[0]
# Convert cook time to minutes


    hours = int(cook_time_str[0]) if cook_time_str[0] else 0
    minutes = int(cook_time_str[1]) if cook_time_str[1] else 0
    cook_time_minutes = hours * 60 + minutes

    hours = int(prep_time_str[0]) if prep_time_str[0] else 0
    minutes = int(prep_time_str[1]) if prep_time_str[1] else 0
    prep_time_minutes = hours * 60 + minutes


    hours = int(total_time_str[0]) if total_time_str[0] else 0
    minutes = int(total_time_str[1]) if total_time_str[1] else 0
    total_minutes = hours * 60 + minutes
    #total = re.findall(r'Total:\s*(\d+)\s*mins', lines[0])[0]
    #servings = re.findall(r'Servings?:\s*(\d+?)\s*(?:to\s*\d+)?\s*(?:servings|ratings)', lines[0])[0] # sometimes instead of saying servings 6 they say servings 6 to 8 in this case we make it servings 6
    #servings = re.findall(r'servings?:\s*(\d+?)\s*(?:to\s*\d+)?\s*(?:servings?|ratings)', lines[0], re.IGNORECASE)[0]
    #text = "The serving size is 3 servings per container."
    
    if(lines[0].count('serv')):
        match = re.search(r'serv\w*:\D*(\d+)', lines[0], re.IGNORECASE)
        if match:
            servings=(match.group(1))
    else:
        servings=1

    

    # Create a pandas DataFrame from the extracted data
    df = pd.DataFrame({
        'Prep': [prep_time_minutes],
        'Cook': [cook_time_minutes],
        'Total': [total_minutes],
        'Servings': [servings]
    })
    df = df.astype(int)
    return df

now for the second block of information which contains the rating for the dish 
this one was also messy because instead of writing a number they only display the star rating with 0.5 increments so we needed to count how many full stars and half stars there are 

In [8]:
def get_info2_fast(soup_obj):    
    soup = soup_obj
    results_items = soup.find_all(class_='comp js-feedback-trigger aggregate-star-rating mntl-block')    
    #print(results_items.prettify())
    for item in results_items:##result items size is 1
        text=item.prettify()
        full_stars=text.count('class="active"')
        half_stars=text.count('class="half"')
        return(full_stars+0.5*half_stars)

up next is the rating count nothing to special about this one 

In [9]:
def get_rating_count(soup_obj):
    soup = soup_obj
    rating_elements = soup.find_all("div", attrs={'class': "comp aggregate-star-rating__count mntl-aggregate-rating mntl-text-block"})
    for rating_element in rating_elements:
        rating_text = rating_element.text.strip()
        try:
            num_ratings = int(rating_text.split()[0])
            return num_ratings
        except ValueError:
            pass
    return 0

up next we have the 3rd block of information which gives us the nutritional values there was an issue here that alcoholic beverages had no nutrinal value and the site had a lot of those

In [10]:
def get_info3_fast(soup_obj):
    soup = soup_obj
    results_items = soup.find_all(class_='nutrition-info__table--row')

    nutritional_vals=[]    


    for item in results_items:
        nutritional_vals.append(item.text.strip())
    new_list = []
    for s in nutritional_vals:
        # Split the string by the \n character and add the two parts to a new list
        parts = s.split('\n')
        # Add the new strings to the new list in the desired format
        #[caleories:934,fat:134g,carbs:999]
        new_list.extend([parts[1], parts[0]])
    #[calories,934,far,134g,carbs,1123,]
    df = pd.DataFrame({'nutrient': new_list[::2], 'value': new_list[1::2]})

    # Set 'nutrient' column as index and transpose DataFrame
    df = df.set_index('nutrient').T
    if df.empty: #recepies like cocktails have no calories

        df = pd.DataFrame(columns=['nutrient', 'Calories', 'Fat', 'Carbs', 'Protein'])

        # add a row filled with zeros
        df.loc[0] = ['value', 0, '0g', '0g', '0g']
        df['Calories'] = df['Calories'].astype(np.int32)
        return df 

    df['Calories'] = df['Calories'].astype(int)

    return df

next is the 4th block of relevant information which holds the list of ingridients this was also not fun as the class names for some receipies changed and werent consistent resulting in a lot of hotfixed and hours of debugging which is why we added cond to check if its the old varient of the site or not

In [11]:
def get_info4_fast(soup_obj):
    cond=0
    soup = soup_obj
    results_items = soup.find_all(class_='structured-ingredients__list text-passage')
                                        #comp ingredient-list simple-list simple-list--bulleted  
    #print(results_items)                                           
    if(results_items==[]): #sometimes they like to change the class name
        soup = soup_obj
        results_items = soup.find_all(class_='simple-list__item js-checkbox-trigger ingredient text-passage')
        cond=1
    nutritional_vals=[]
    if(results_items==[]):
        return []
    
    final_lst=[]

    for item in results_items:    
        nutritional_vals.append(item.text.strip())
        #print(item.text.strip())
    if(cond==1):
        return nutritional_vals
    else:        
        for i in nutritional_vals:
            my_list = [s.strip() for s in i.split('\n\n\n')]
            final_lst.extend(my_list)
            #print(final_lst)
        
        return(final_lst)

we also wanted to add some info of our own so we looked at the ingridients and tried to figure out if its dairy meat fur or dairy and meat as for keywords we just googled common meat or common dairy products and coppied the first few

In [12]:
def analyze_recipe(ingredients):
    dairy_keywords = ["milk", "cheese", "yogurt", "cream", "butter", "whey", "casein", "curds"]
    meat_keywords = ["beef", "chicken", "pork", "lamb", "turkey", "venison", "duck", "bacon", "sausage",
                     "ham", "prosciutto", "pepperoni", "salami", "chorizo", "bresaola", "pastrami",
                     "corned beef", "veal", "goose", "game", "elk", "bison", "rabbit", "boar", "guinea fowl", "quail"]
    
    categories = {'Dairy': False, 'Meat': False, 'Fur': True}
    
    for ingredient in ingredients:
        ingredient = ingredient.lower()
        if any(keyword in ingredient for keyword in dairy_keywords):
            categories['Dairy'] = True
            categories['Fur'] = False
        elif any(keyword in ingredient for keyword in meat_keywords):
            categories['Meat'] = True
            categories['Fur'] = False
            
    df = pd.DataFrame(categories, index=[0])
    return df

now we want to merge all of this data into a single row of a dataframe and again we will be looping over this one 

In [13]:
def merge_fast(url,recepie_name):
    df = pd.DataFrame(columns=['Name','Prep', 'Cook', 'Total', 'Servings', 'Rating','Rating_Count','Dairy','Meat','Fur', 'Calories', 'Fat', 'Carbs', 'Protein', 'Ingredients'])
    #Dairy
    #Meat
    #Fur
    soup_obj= load_soup_object(url)

    recipe_df = get_info1_fast(soup_obj)

    ratings_list=get_info2_fast(soup_obj)

    nutrition_df = get_info3_fast(soup_obj)

    ingredients = get_info4_fast(soup_obj)

    rating_num = get_rating_count(soup_obj)

    meatdaity_df =analyze_recipe(ingredients)

    if(ingredients==[]):
        ingredients=['','','','']
        print(type(nutrition_df['Calories'][0]))
    new_row = {
        'Name':recepie_name,
        'Prep': recipe_df['Prep'][0],
        'Cook': recipe_df['Cook'][0],
        'Total': recipe_df['Total'][0],
        'Servings': recipe_df['Servings'][0],
        'Rating': ratings_list,
        'Rating_Count':rating_num,
        'Dairy':meatdaity_df['Dairy'][0],
        'Meat':meatdaity_df['Meat'][0],
        'Fur':meatdaity_df['Fur'][0],
        'Calories': nutrition_df['Calories'][0],
        'Fat': nutrition_df['Fat'][0],
        'Carbs': nutrition_df['Carbs'][0],        
        'Protein': nutrition_df['Protein'][0],
        'Ingredients': [ingredients]
    }
    #print(new_row)

    # add the new row to the DataFrame
    df = pd.concat([df, pd.DataFrame(new_row)], ignore_index=True)
    #df_concat = pd.concat([df1, df2], keys=['df2'])
    df['Prep'] = df['Prep'].astype(float)
    df['Cook'] = df['Cook'].astype(float)
    df['Total'] = df['Total'].astype(float)
    df['Servings'] = df['Servings'].astype(float)
    df['Rating'] = df['Rating'    ].astype(float)
    df['Calories'] = df['Calories'].astype(float)

    # display the updated DataFrame
    #print("another line added")
    return(df)

and now we want to do this for each receipe that was found up until now here comes another function :)
it will take a while so again to speed up work later we are saving it to a file 

In [22]:
def fast_scrape(csv_file_name):
    recipe_names = []
    recipe_links = []
    count=0

    with open(csv_file_name, encoding='utf-8', newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            recipe = ','.join(row).strip().replace('\x9c', '')  # Join recipe name and link with a comma, remove problematic characters
            last_comma = recipe.rfind(',')  # Find the index of the last comma in the recipe string
            if last_comma != -1:
                recipe_name = recipe[:last_comma].strip()  # Get the recipe name before the last comma
                recipe_link = recipe[last_comma+1:].strip()  # Get the recipe link after the last comma
                recipe_names.append(recipe_name)
                recipe_links.append(recipe_link)
            else:
                print(f"Invalid row: {row}")

    final_df=pd.DataFrame()
    for name, link in zip(recipe_names, recipe_links):
        # print(f"Recipe name: {name}")
        # print(f"Recipe link: {link}")
        temp_df=merge_fast(link,name)
        temp_df.set_index('Name', inplace=True)  # set the index of temp_df to the recipe name
        if(final_df.empty):
            final_df=temp_df
        else:
            final_df = pd.concat([final_df,temp_df])
        print(final_df)

    #final_df.to_pickle('dataframe.pkl')
    #final_df.to_pickle('dataframe.pkl', protocol=4, encoding='utf-8')
    final_df.to_csv('my_data.csv', index=True, encoding='utf-8')
    print(final_df)


In [28]:
fast_scrape('Recipe_Links_and_Names.csv')

MissingSchema: Invalid URL 'Recipe_link': No scheme supplied. Perhaps you meant http://Recipe_link?

now that we have our dataframe lets see what we are working with we will try to visualize the data and clean it up later ofc we want some functions we will make functions for scatter plot 
histogram and a 1 column pie

In [21]:
def draw_scatter_2_params(col_name_1,col_name_2):
    df=pd.read_csv('my_data.csv')
    df['Fat'] = df['Fat'].str.replace('g', '').str.replace(',', '').astype(int)
    df['Carbs'] = df['Carbs'].str.replace('g', '').str.replace(',', '').astype(int)
    df['Protein'] = df['Protein'].str.replace('g', '').str.replace(',', '').astype(int)
    df.plot.scatter(x=col_name_1, y=col_name_2)
    
    plt.show()

In [20]:
def draw_histo_1_params(col_name):
    # read in your dataframe from a csv file
    df = pd.read_csv('my_data.csv')
    df['Fat'] = df['Fat'].str.replace('g', '').str.replace(',', '').astype(int)
    df['Carbs'] = df['Carbs'].str.replace('g', '').str.replace(',', '').astype(int)
    df['Protein'] = df['Protein'].str.replace('g', '').str.replace(',', '').astype(int)
    # choose the column you want to use for the histogram

    # sort the column values into bins
    bin_values, bin_edges = np.histogram(df[col_name], bins='auto')

    # create the histogram using the sorted bins
    plt.hist(df[col_name], bins=bin_edges)

    # add labels and title to the histogram
    plt.xlabel('Values')
    plt.ylabel('Frequency')
    plt.title('Histogram of ' + col_name)

    # display the histogram
    plt.show()

In [19]:
def draw_pie_1_params(col_name):
    df = pd.read_csv('my_data.csv')
    df['Fat'] = df['Fat'].str.replace('g', '').str.replace(',', '').astype(int)
    df['Carbs'] = df['Carbs'].str.replace('g', '').str.replace(',', '').astype(int)
    df['Protein'] = df['Protein'].str.replace('g', '').str.replace(',', '').astype(int)
    # choose the column you want to use for the pie chart    

    # get the count of unique values in the column
    value_counts = df[col_name].value_counts()

    # create the pie chart
    plt.pie(value_counts.values, labels=value_counts.index, autopct='%1.1f%%')

    # add title to the pie chart
    plt.title('Pie Chart of ' + col_name)

    # display the pie chart
    plt.show()

in addition we wanted a pie chart of how much is meat dairy meat and dairy and fur

In [18]:
def draw_pie_meat_dairy_fur():
    df = pd.read_csv('my_data.csv')
    
    # Calculate the number of recipes in each category
    meat_count = len(df[df['Meat'] == True])
    dairy_count = len(df[df['Dairy'] == True])
    fur_count = len(df[df['Fur'] == True])
    dairy_meat_count = len(df[(df['Dairy'] == True) & (df['Meat'] == True)])

    # Create a list of category counts and labels
    counts = [meat_count, dairy_count, fur_count, dairy_meat_count]
    labels = ['Meat', 'Dairy', 'Fur', 'Dairy&Meat']

    # Create the pie chart
    plt.pie(counts, labels=labels, autopct='%1.1f%%')

    # Add a title to the chart
    plt.title('Recipe Categories')

    # Show the chart
    plt.show()

lets run all of these and see if we can already draw some conclusions